### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
# Calculate the total number of schools
total_schools_array = school_data_complete["school_name"].unique()
total_schools = len(total_schools_array)

# Calculate the total number of students
total_students_array = school_data_complete["student_name"].value_counts()
total_students = sum(total_students_array)

# Calculate the total budget
total_budget = school_data["budget"].sum()

# Calculate the average math score
average_math = school_data_complete['math_score'].mean()

# Calculate the reading math score
average_reading = school_data_complete['reading_score'].mean()

# Calculate the percentage of students with a passing math score (70 or greater)
# Filtering the DF with a conditional
school_data_filtered = school_data_complete[school_data_complete['math_score'] >= 70]
percent_passing_math = school_data_filtered['Student ID'].count() / total_students

# Calculate the percentage of students with a passing reading score (70 or greater)
school_data_filtered_2 = school_data_complete[school_data_complete['reading_score'] >= 70]
percent_passing_reading = school_data_filtered_2['Student ID'].count() / total_students
percent_passing_reading

# Calculate the percentage of students who passed math and reading (% Overall Passing)
school_data_filtered_3 = school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)]
school_data_filtered_3
percent_passing_overall = school_data_filtered_3['Student ID'].count() / total_students

# Create a dataframe to hold the above results
results_df = pd.DataFrame({"Total Schools":total_schools, "Total Students": total_students, "Total Budget": total_budget, "Average Math Score": average_math, "Average Reading Score": average_reading, "% Passing Math": percent_passing_math, "% Passing Reading": percent_passing_reading, "% Overall Passing": percent_passing_overall}, index=[0])
# Format items
results_df["Total Students"] = results_df["Total Students"].map("{:,}".format)
results_df["Total Budget"] = results_df["Total Budget"].map("${:,.2f}".format)
results_df["% Passing Math"] = results_df["% Passing Math"] * 100
results_df["% Passing Reading"] = results_df["% Passing Reading"] * 100
results_df["% Overall Passing"] = results_df["% Overall Passing"] * 100
results_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [18]:
# Building the Data Frame
sdc_gp = school_data_complete.groupby("school_name")
sd_copy = school_data.copy()
sd_df = sd_copy.set_index("school_name").drop(columns="School ID").sort_values(by="school_name", ascending=True)
sd_df["Per Student Budget"] = sd_df["budget"] / sd_df["size"] 
sd_df[["Average Math Score", "Average Reading Score"]] = sdc_gp.mean()[["math_score", "reading_score"]]

# Calculating the harder elements on a seperate DF
my_filter_math = school_data_complete["math_score"] >= 70
my_filter_reading = school_data_complete["reading_score"] >= 70
sg_math = school_data_complete[my_filter_math].groupby("school_name")["math_score"]
sg_reading = school_data_complete[my_filter_reading].groupby("school_name")["reading_score"]

percents_df = pd.DataFrame({
    "% Passing Math": sg_math.count() / sdc_gp.count()["type"]*100,
    "% Passing Reading": sg_reading.count() / sdc_gp.count()["type"]*100,
    "% Overall Passing": school_data_complete[(my_filter_math) & (my_filter_reading)] \
        .groupby("school_name") \
        .count()["reading_score"] / \
        sdc_gp.count()["type"]*100
})


# Merging the 2 DF's and formatting
school_summary = sd_df.merge(percents_df, on=["school_name"])
school_summary = school_summary.rename(columns={
    "type": "School Type", 
    "size": "Total Students", 
    "budget": "Total School Budget"
})
budget_unformat = school_summary[["Total School Budget", "Per Student Budget"]]
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [102]:
# Initial attempt - keeping code commented out as reference
# gp = school_data_complete.groupby(["school_name"])
# student_count = gp.count()["type"]
# average_scores = gp.mean()[["reading_score", "math_score"]]
# data = [student_count, average_scores["math_score"], average_scores["reading_score"]]
# df = pd.DataFrame({"Avg Math Score": average_scores["math_score"], "Avg Reading Score": average_scores["reading_score"]}, index=gp.indices)
# df.reset_index(inplace=True)
# df = df.rename(columns={"index": "school_name"})
# school_data_ext = school_data.merge(df, on=["school_name"])
# school_data_ext = school_data_ext.sort_values(by="school_name", ascending=True)
# school_data_ext["Per Student Budget"] = school_data_ext["budget"] / school_data_ext["size"]
# school_data_ext = school_data_ext.drop(columns="School ID")
# school_data_ext = school_data_ext[["school_name","type","size","budget","Per Student Budget","Avg Math Score","Avg Reading Score"]]
# student_data_math_df = school_data_complete[school_data_complete["math_score"]>=70]
# grouped_by = student_data_math_df.groupby(["school_name"])
# school_data_ext["% Passing Math"] = grouped_by.count()["math_score"] / school_data_complete["school_name"].value_counts()
# school_data_ext

## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [5]:
ss_copy = school_summary.copy()
top_performing = ss_copy.sort_values(ascending=False, by="% Overall Passing").head(5)
top_performing

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Total School Budget Format,Per Student Budget Format
school_name,,,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769,"$1,081,356.00",$582.00
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012,"$1,043,130.00",$638.00
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455,"$917,500.00",$625.00
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567,"$1,319,574.00",$578.00
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541,"$585,858.00",$609.00


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [6]:
ss_copy = school_summary.copy()
bottom_performing = ss_copy.sort_values(ascending=True, by="% Overall Passing").head(5)
bottom_performing

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Total School Budget Format,Per Student Budget Format
school_name,,,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247,"$2,547,363.00",$637.00
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476,"$1,884,411.00",$639.00
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884,"$1,910,635.00",$655.00
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508,"$3,022,020.00",$652.00
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172,"$3,094,650.00",$650.00


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
sdc_gp = school_data_complete.groupby(["school_name", "grade"])
math_df = sdc_gp.mean()["math_score"].reset_index()
math_df_9 = math_df[math_df["grade"]=="9th"].set_index("school_name")["math_score"]
math_df_10 = math_df[math_df["grade"]=="10th"].set_index("school_name")["math_score"]
math_df_11 = math_df[math_df["grade"]=="11th"].set_index("school_name")["math_score"]
math_df_12 = math_df[math_df["grade"]=="12th"].set_index("school_name")["math_score"]
math_score_df = pd.DataFrame({
    "9th": math_df_9,
    "10th": math_df_10,
    "11th": math_df_11,
    "12th": math_df_12
})
math_score_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [10]:
sdc_gp = school_data_complete.groupby(["school_name", "grade"])
reading_df = sdc_gp.mean()["reading_score"].reset_index()
reading_df_9 = reading_df[reading_df["grade"]=="9th"].set_index("school_name")["reading_score"]
reading_df_10 = reading_df[reading_df["grade"]=="10th"].set_index("school_name")["reading_score"]
reading_df_11 = reading_df[reading_df["grade"]=="11th"].set_index("school_name")["reading_score"]
reading_df_12 = reading_df[reading_df["grade"]=="12th"].set_index("school_name")["reading_score"]
reading_score_df = pd.DataFrame({
    "9th": reading_df_9,
    "10th": reading_df_10,
    "11th": reading_df_11,
    "12th": reading_df_12
})
reading_score_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [22]:
# Establish the spending bins and group names.
ssc = school_summary.copy(deep=True)
spending_bins = [0, 585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]
# Categorize spending based on the bins.
per_student = budget_unformat.loc[:,"Per Student Budget"].astype(float)
ssc["Spending Ranges (Per Student)"] = pd.cut(per_student, spending_bins, labels=group_names)
ssc = ssc.groupby("Spending Ranges (Per Student)").mean()[[
    "Average Math Score", "Average Reading Score","% Passing Math",	"% Passing Reading", "% Overall Passing"
]]
ssc
# Formating
ssc.head(4).style.format({
    "Average Math Score": "{:.2f}",
    "Average Reading Score": "{:.2f}",
    "% Passing Math": "{:.2f}",
    "% Passing Reading": "{:.2f}",
    "% Overall Passing": "{:.2f}"
})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [155]:
# Establish the spending bins and group names.
ssc = school_summary.copy(deep=True)
spending_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
# Categorize spending based on the bins.
score_per_school = school_summary.loc[:,"Total Students"].astype(float)
ssc["Spending Ranges (Per Student)"] = pd.cut(score_per_school, spending_bins, labels=group_names)
ssc = ssc.groupby("Spending Ranges (Per Student)").mean()[[
    "Average Math Score", "Average Reading Score","% Passing Math",	"% Passing Reading", "% Overall Passing"
]]
ssc

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [164]:
# My own notes, ignore
school_summary.loc[["Bailey High School", "Figueroa High School"],["School Type", "Total Students"]]

,School Type,Total Students
school_name,,
Bailey High School,District,4976
Figueroa High School,District,2949


## Scores by School Type

* Perform the same operations as above, based on school type

In [16]:
scores_by_school = school_summary.groupby("School Type").mean()[
    ["Average Math Score",	"Average Reading Score",	"% Passing Math",	"% Passing Reading",	"% Overall Passing"]]
scores_by_school

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
